In [1]:
import numpy as np
import glob

%matplotlib qt
import matplotlib.pyplot as plt
import matplotlib as mp
import pandas as pd
import os

In [2]:
#set up colors for plots
import matplotlib.colors as colors
cmap = plt.get_cmap('terrain')
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap
new_cmap = truncate_colormap(cmap, 0.0, 0.75)
collist = [new_cmap(k) for k in np.linspace(0, 1, 6)]

In [3]:
import dill

with open('smooth_pos_v8.pkl', 'rb') as var:
    sm_pos = dill.load(var)
    
with open('smooth_vel_v8.pkl', 'rb') as var:
    sm_vel = dill.load(var)

with open('smooth_acc_v8.pkl', 'rb') as var:
    sm_acc = dill.load(var)
    
with open('smooth_times_v8.pkl', 'rb') as var:
    d_times = dill.load(var)
    
with open('raw_dictionaries_v9.pkl', 'rb') as var:
    all_imported = dill.load(var)

### Trial metadata

In [4]:
#build gap sizes, snake number, trial number lists
gs_svl = []
sn_id = []
trial_ns = []
herz = []
    
for i in np.arange(len(all_imported)):
    
    d = all_imported[i]
    gs_svl.append(d['gs_%'])
    sn_id.append(d['ID'])
    trial_ns.append(d['tn'])
    herz.append(d['fr'])

# Generate landing velocities
## Linear fit to the last few frames in the trial. 
### Step 1: determine how many frames to use by looking at residuals

### Review residuals and velocities for all trials

In [5]:
fig,ax=plt.subplots(2,3)
dims = ['X','Y','Z']

for n in np.arange(len(sm_pos)):
    test_snake = sm_pos[n][:,0,:]
    fr = herz[n]
    t_vals = [each/fr for each in np.arange(np.shape(test_snake)[0])]
    #     vals_try = [3,4,5,6,7,8,9,10,20,30] I tried these first, and then refined to the next set (vals_try1)
    vals_try1= [9,10,11,12,13,14,15,16,17,18]
    
    for d in [0,1,2]:
        res_snake = test_snake[:,d]  
    
        for i in vals_try1:
            v_line, residuals, rank, singular_values, rcond = np.polyfit(t_vals[-i:],res_snake[-i:],1,full=True)
            vel_pts = [v_line[1]+v_line[0]*each for each in t_vals]

            if n==0:
                thing = str(i)
            else:
                thing = '_no legend_'
            ax[0,d].scatter(n,residuals[0]/1000.0,label=thing,s=5)
            ax[1,d].scatter(n,v_line[0]/1000.0,s=5)
            ax[1,d].set_xlabel(dims[d])

ax[0,0].set_ylabel('residual, m')
ax[1,0].set_ylabel('velocity, m/s')
handles, labels = ax[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc=(0.25,0.8888),ncol=len(vals_try1))

#based on this plot, using last 0.07 of a second for the landing velocity

### Based on above, using 0.07 seconds as "landing velocity window"
### Step 2) generate the landing velocity  list

In [6]:
land_vs1 = []

for n in np.arange(len(sm_pos)):
    test_snake = sm_pos[n][:,0,:]
    fr = herz[n]
    t_vals = [each/fr for each in np.arange(np.shape(test_snake)[0])]
    i = int(fr*0.07)
    vel_vals = []
    
    for d in [0,1,2]: #linear fit to each dimension for the final 0.07s
        res_snake = test_snake[:,d] 
        v_line, residuals, rank, singular_values, rcond = np.polyfit(t_vals[-i:],res_snake[-i:],1,full=True)
        vel_pts = [v_line[1]+v_line[0]*each for each in t_vals]
        vel_vals.append(v_line[0]/1000.0) #convert to m/s from mm/s
        
    landv = np.linalg.norm(vel_vals)
    land_vs1.append(landv)

# Generate resultant velocity

Takes the X, Y, and Z values for each marker and each frame and creates the resultant velocity value. 
Creates a list where each entry is the array of resultant velocity values for a given trial

Then, examine data for outliers. 

Then, remove outliers and save new data

In [7]:
#resultant velocity (head marker) for each trial
sm_vres = []
for each in sm_vel:
    each = each/1000 #change mm to m
    resultant = np.linalg.norm(each,axis=2) #resultant velocities for each frame and marker   
    sm_vres.append(resultant[:,0]) #save the head marker

In [8]:
#plot resultant velocity graphs
col = 13
row = 14

fig, ax = plt.subplots(col,row,sharey=True)

for i in np.arange(col):
    for t in np.arange(row):
        n = i+col*t
        #print(n)
        ax[i,t].set_title(str(all_imported[n]['tn']),size=10, y=0.5)
        ax[i,t].plot(sm_vres[n])
        ax[i,t].axis('off')
        
fig.suptitle('all head velocity data (n= '+str(len(sm_vres))+'), smoothed')

Text(0.5, 0.98, 'all head velocity data (n= 182), smoothed')

# Generate "moving" velocity

Average resultant velocity for the frames in which the snake is moving (i.e. res_V is >threshold)


## Determine non-moving threshold
### Compare trials where snake has significant non-moving period. 
#### 74 - not moving until frame 14920
#### 243 - not moving util frame 3300
#### 54 - moving whole time

In [9]:
x0 = trial_ns.index(74)
x1 = trial_ns.index(243)
x2 = trial_ns.index(54)

frames0 = list(all_imported[x0]['fn'])
frames1 = list(all_imported[x1]['fn'])
frames2 = list(all_imported[x2]['fn'])

ind0 = frames0.index(14920)
ind1 = frames1.index(3300)

In [10]:
notmoving = sm_vel[x0][:ind0,:,:]
notmoving1 = sm_vel[x1][:ind1,:,:]
moving = sm_vel[x2]

In [11]:
res = np.linalg.norm(notmoving,axis=2)
res1 = np.linalg.norm(notmoving1,axis=2)
res2 = np.linalg.norm(moving,axis=2)

In [12]:
import matplotlib as mpl
colormaps = [plt.get_cmap('Blues'),plt.get_cmap('Greens'),plt.get_cmap('PuRd')]
plots = [res,res1,res2]
labels = ['Not Moving','Not Moving','Moving']
fig,ax = plt.subplots()

for i in [0,1,2]:
    colormap = colormaps[i]
    colorlist = [colormap(k) for k in np.linspace(0, 1, 10)]
    custom_cycler = mpl.cycler(color = colorlist)
    ax.set_prop_cycle(custom_cycler)
    ax.plot(plots[i],label=labels[i])

ax.set_ylabel('Velocity (mm/s)')
ax.set_xlabel('Frame number')

Text(0.5, 0, 'Frame number')

### Based on above, threshold = 0.025 m/s

In [13]:
thresh = 0.025
avg_rxs = []

for trl in np.arange(len(all_imported)):
    resv = sm_vres[trl]
    A = len(resv)

    #get the average resultant velocity over frames where the snake is actually moving forward.
    vx = sm_vel[trl][:,0,0]/1000.0 #change mm/s to m/s
    move_x = [resv[t] for t in np.arange(A) if vx[t]>thresh] #select frames where horizontal/forward component is greater than threshold.
    avg_rx = np.nanmean(move_x) #average resultant head speed across all such frames.
    
    avg_rxs.append(avg_rx)

In [14]:
#max resultant head velocity for all trials
max_res = []

for i in np.arange(len(sm_vres)):
    res = sm_vres[i]    
    max_res.append(np.nanmax(res)) #maximum resultant head velocity

# Create dataframe

In [15]:
vdf = pd.DataFrame(list(zip(trial_ns,gs_svl,sn_id)),columns = ['tn','gsr','ID'])

In [16]:
cwd = os.getcwd()
bpd = pd.read_csv(cwd+'/R files/Summary Datasets/bdata.csv',index_col=None)

In [17]:
#add svls and behaviors
svls = [all_imported[ind]['svl']/100 for ind in np.arange(0,len(all_imported))]
vdf['svl'] = svls
vdf['beh'] = bpd['beh']

### Add speed values

In [18]:
vdf['mhv'] = max_res #max head speed
vdf['landv'] = land_vs1 #landing speed
vdf['axv'] = avg_rxs #average forward moving speed

## Relative Velocities

In [19]:
#scale all velocities to SVL/sec speeds. 
A,B = np.shape(vdf)
scale_lv = []
scale_mv = []
scale_av = []
for i in np.arange(A):
    trial = vdf['tn'][i]
    ind = trial_ns.index(trial)
    svl = svls[i]
    lv = vdf['landv'][i]
    mv = vdf['mhv'][i]
    av = vdf['axv'][i]
    
    scale_lv.append(lv/svl) #change m/s to svls/sec
    scale_mv.append(mv/svl)
    scale_av.append(av/svl)

In [20]:
vdf['slv'] = scale_lv #scaled landing velocity
vdf['smv'] = scale_mv #scaled max velocity
vdf['sav'] = scale_av #scaled average vleocity (when moving forward)

In [21]:
#save as csv for analysis in R
vdf.to_csv(cwd+'/R files/Summary Datasets/vdata.csv')

# Statistical Analyses

In [22]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.optimize import curve_fit
from scipy.stats.distributions import  t

## What is the relationship between gap size and velocity metrics?

In [23]:
#all together
import seaborn as sns
IDs = [95,88,90,89,94,85]
shapes = ['<','v','s','^','>','o']

collist = sns.color_palette("terrain", 6)
maxV = np.array(vdf['smv'])
avgV = np.array(vdf['sav'])
landV = np.array(vdf['slv'])
gsr = np.array(vdf['gsr'])
sneks = np.array(vdf['ID'])
behs = np.array(vdf['beh'])

fig, ax = plt.subplots(3,sharex=True,sharey=True)

ax[0].set_ylabel("Average (SVL/s)")
ax[1].set_ylabel("Max (SVL/s)")
ax[2].set_ylabel("Landing (SVL/s)")
ax[2].set_xlabel("Gap size (%SVL)")


for w in np.arange(len(IDs)):
    X = gsr[sneks==IDs[w]]
    Y2 = landV[sneks==IDs[w]]
    Y0 = avgV[sneks==IDs[w]]
    Y1 = maxV[sneks==IDs[w]]
    B = behs[sneks==IDs[w]]

    color =  np.reshape(np.array(collist[w]),(1,3))

    for i in np.arange(len(X)):
        if B[i] == 0:
            ec = np.reshape(np.array(color),(1,3))
            fc = 'None'
            size = 50
        else:
            fc = np.reshape(np.array(color),(1,3))
            ec=None
            size = 50


        ax[2].scatter(X[i], Y2[i], marker = shapes[w], label=str(IDs[w]), s=50,edgecolor=ec,facecolor=fc)
        ax[1].scatter(X[i], Y1[i], marker = shapes[w], label=str(IDs[w]), s=50,edgecolor=ec,facecolor=fc)
        ax[0].scatter(X[i], Y0[i], marker = shapes[w], label=str(IDs[w]), s=50,edgecolor=ec,facecolor=fc)

In [24]:
# Graph comparing absolute and relative values
fig,ax = plt.subplots(2,3, sharex=True,sharey=True)
fig.suptitle('Resultant head velocity (m/s)')

ax[0,0].set_title('Max')
ax[0,1].set_title('Average, moving forward (x)')
ax[0,2].set_title('Landing Velocity')

ax[0,0].set_ylabel('Absolute (m/s)')
ax[1,0].set_ylabel('Relative (svl/s)')

ax[1,0].set_xlabel('Gap size (%SVL)')
ax[1,1].set_xlabel('Gap size (%SVL)')
ax[1,2].set_xlabel('Gap size (%SVL)')

color_labels = vdf['ID'].unique()
Norm = mp.colors.Normalize(vmin=0,vmax=5)
rgbs = [new_cmap(Norm(i)) for i in np.arange(6)]
color_map = dict(zip(color_labels, rgbs))

ax[0,0].scatter(vdf['gsr'],vdf['mhv'],c=vdf['ID'].map(color_map),s=10)
ax[1,0].scatter(vdf['gsr'],vdf['smv'],c=vdf['ID'].map(color_map),s=10)


ax[0,1].scatter(vdf['gsr'],vdf['axv'],c=vdf['ID'].map(color_map),s=10)
ax[1,1].scatter(vdf['gsr'],vdf['sav'],c=vdf['ID'].map(color_map),s=10)

ax[0,2].scatter(vdf['gsr'],vdf['landv'],c=vdf['ID'].map(color_map),s=10)
ax[1,2].scatter(vdf['gsr'],vdf['slv'],c=vdf['ID'].map(color_map),s=10)


### Create initial parameter estimates for sigmoid fits

In [25]:
def sigmoid(x, L, x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return y
def residuals(x,y,p):
    L, x0, k, b = p
    return y - sigmoid(x,L,x0,k,b)

In [26]:
xdata = vdf['gsr'] #relative gap size
ydata = vdf['mhv'] #max head velocity

plt.figure()
plt.scatter(xdata,ydata)

p0 = [max(ydata), np.median(xdata),1,min(ydata)] # this is an mandatory initial guess

pars, pcov = curve_fit(sigmoid, xdata, ydata, p0, method='dogbox')
alpha = 0.05 # 95% confidence interval = 100*(1-alpha)

n = len(ydata)    # number of data points
pn = len(pars) # number of parameters

dof = max(0, n - pn) # number of degrees of freedom

# student-t value for the dof and confidence level
tval = t.ppf(1.0-alpha/2., dof) 

par_names = ['L','x0','k','b']
print('fn: y = L / (1 + e^(-k*(x-x0)))+b')
for i, p, var in zip(range(n), pars, np.diag(pcov)):
    sigma = var**0.5
    low = p - sigma*tval
    high = p+sigma*tval
    print( '''\
    {n} = {est} [{low},  {high}]
    '''.format(n=par_names[i],est=p,low=low,high=high))

xp = np.linspace(20, 120, 500)
pxp = sigmoid(xp,pars[0],pars[1],pars[2],pars[3])

# Plot the results
plt.plot(xp, pxp, '-')
plt.xlabel('Gap Size')
plt.ylabel('Max head velocity') 
plt.grid(True)

print(np.mean(np.abs(residuals(xdata,ydata,pars))))

fn: y = L / (1 + e^(-k*(x-x0)))+b
    L = 1.8255431140235705 [1.657944009009561,  1.99314221903758]
    
    x0 = 57.32151206091584 [55.99605497304828,  58.646969148783406]
    
    k = 0.24754571796723862 [0.1721004999243154,  0.3229909360101618]
    
    b = 0.29195448908716465 [0.1686566784805868,  0.4152522996937425]
    
0.25323091766050404


In [27]:
xdata = vdf['gsr'] #relative gap size
ydata = vdf['landv'] #landing speed

plt.figure()
plt.scatter(xdata,ydata)

p0 = [max(ydata), np.median(xdata),1,min(ydata)] # this is an mandatory initial guess

pars, pcov = curve_fit(sigmoid, xdata, ydata, p0, method='dogbox')
alpha = 0.05 # 95% confidence interval = 100*(1-alpha)

n = len(ydata)    # number of data points
pn = len(pars) # number of parameters

dof = max(0, n - pn) # number of degrees of freedom

# student-t value for the dof and confidence level
tval = t.ppf(1.0-alpha/2., dof) 

par_names = ['L','x0','k','b']
print('fn: y = L / (1 + e^(-k*(x-x0)))+b')
for i, p, var in zip(range(n), pars, np.diag(pcov)):
    sigma = var**0.5
    low = p - sigma*tval
    high = p+sigma*tval
    print( '''\
    {n} = {est} [{low},  {high}]
    '''.format(n=par_names[i],est=p,low=low,high=high))

xp = np.linspace(20, 120, 500)
pxp = sigmoid(xp,pars[0],pars[1],pars[2],pars[3])

# Plot the results
plt.plot(xp, pxp, '-')
plt.xlabel('Gap Size')
plt.ylabel('Max head velocity') 
plt.grid(True)

print(np.mean(np.abs(residuals(xdata,ydata,pars))))

fn: y = L / (1 + e^(-k*(x-x0)))+b
    L = 1.8497368062508106 [1.669320003190718,  2.0301536093109034]
    
    x0 = 57.50273394645001 [56.04680303256113,  58.958664860338885]
    
    k = 0.21699714221264196 [0.1525668120627689,  0.28142747236251503]
    
    b = 0.18783532461132715 [0.05533255083037425,  0.32033809839228006]
    
0.2479515590433792
